# Processamento de Imagem e Visão
## Trabalho Prático 1
### Alunos: Belarmino Sacate (52057) e Miguel Ferreira (51878)

In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

## Funções

Esta função carrega todas as imagens presentes numa pasta expecífica. Ela percorre os ficheiros presentes e seleciona apenas as imagens, através das extenções jpg, png e jpeg.

Cada imagem é guardada num dicionário onde a chave é o nome do ficheiro e o valor é a imagem em si. Assim, consegui-mos aceder a todas as imagens facilmente.

In [5]:
def carregar_imagens(pasta):

    #dicionario para guardar as imagens
    imagens = {}
    
    for nome_ficheiro in os.listdir(pasta):
        if nome_ficheiro.lower().endswith(('.jpg', '.png', '.jpeg')):
            caminho = os.path.join(pasta, nome_ficheiro)
            imagem = cv2.imread(caminho)
            imagens[nome_ficheiro] = imagem
    return imagens

Nesta função a imagem é convertida do formato BGR (padrão usado pelo OpenCV), para o formato RGB. A função utilizada é cv2.cvtColor com a imagem e conversão pretendida.

In [6]:
def converter_para_rgb(imagem):
    return cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

A função refinar_mascara() usa vários operadores morfológicos para melhorar a qualidade da máscara binária gerada.

É usado um elemento estruturante, ou seja, uma matriz pequena, com a sua forma e tamanho diferente, que determina a transformação efetuada.
Foi escolhida a morfologia elíptica para detetar melhor as peças irregulares, mantendo o formato dos objetos. Apesar de a maior parte das peças serem retângulares, conseguiu-se ajustar os vários operadores de forma à não prejudicar estas peças.

Em primeiro lugar, foi utilizado o MORPH_CLOSE, que é uma dilatação sucedida de uma erosão, com o objetivo de fechar pequenos buracos nas peças de lego. Estes buracos surgiram principalmente nas peças com a cor mais próxima do fundo azul. 

De seguida usou-se o MORPH_OPEN, ou seja, uma erosão seguida de uma dilatação, para remover pequenos defeitos nos contornos dos objetos ou falsos positivos detetados na imagem.

Finalmente, foi feita uma erosão e dilatação com matrizes maiores com o objetivo de alimar todas as peças detetadas, suavizando os contornos, tornando-as mais fáceis de classificar.

In [7]:
def refinar_mascara(mascara):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    kernel2= cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    kernel3= cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (30, 30))
    kernel4= cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))

    mascara_close = cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, kernel1)
    mascara_open = cv2.morphologyEx(mascara_close, cv2.MORPH_OPEN, kernel2)
    mascara_erosao = cv2.erode(mascara_open, kernel3)
    mascara_final = cv2.dilate(mascara_erosao, kernel4)

    return mascara_final

A função seguinte aplica a máscara binária gerada na imagem original através da operação bitwise. Nesta operação, os pixeis com valor 0, correspondentes à área preta na imagem, tornam-se pretos na imagem resultante. Os pixeis branco mantêm as cores da imagem originais.

In [8]:
def aplicar_mascara(imagem, mascara):
    return cv2.bitwise_and(imagem, imagem, mask=mascara)

Este método é responsável por guardar as imagens no computador. Os resultados foram colocados na pasta do mesmo nome.

In [9]:
def gravar_imagem(imagem, nome_arquivo):
    cv2.imwrite(nome_arquivo, imagem)

Esta função cria uma máscara binária a partir dos limites de cores definidos. É utilizado o cv2.inRange(), que irá transformar as cores dentro do intervalo fornecido. Estas são transformadas em preto (0). As cores restantes que correspondem às peças ficam brancas (1).

In [10]:
def criar_mascara_por_cor(imagem_rgb, cor_inferior, cor_superior):
    mascara = cv2.inRange(imagem_rgb, cor_inferior, cor_superior)
    return mascara

Esta função permite ver a imagem numa janela à parte, esperando que uma tecla seja pressionada antes de fechar a própria janela.

In [11]:
def exibir_imagem_cv2(titulo, imagem):
    cv2.imshow(titulo, imagem)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Esta função calcula algumas propriedades para cada objeto identificado através do seu contorno. 

Primeiramente, obtemos a área de um objeto detetado na imagem, que pode ser uma peça de Lego ou não. De seguida, aplica-se o método minAreaRect() que gera o retângulo mínimo que consegue preencher o objeto todo. 

De seguida, é calculado o rácio entre a área real e o retângulo mínimo. Como o objetivo é identificar peças de Lego, as quais são retângulares, as duas áreas serão muito parecidas. Assim, conseguimos separar peças de Lego e outros objetos irregulares através da comparação destas duas áreas. 

Por fim, é calculada a razão entre a altura e largura da peça. Para garantir que as proporções possam ser comparadas corretamente, é usado um "if" que garante que o valor seja sempre maior ou igual a 1.

In [1]:
def extracao_propriedades(contorno):
    
    #area do contorno da peça
    area = cv2.contourArea(contorno)

    #retangulo minimo
    rect = cv2.minAreaRect(contorno)
    box = cv2.boxPoints(rect)
    box = np.int32(box)
    
    #racio de area
    area_rect = cv2.contourArea(box)
    if area_rect == 0:
        racio = 0
    else:
        racio = (area / area_rect) * 100
        
    (x, y), (w, h), a = rect
    
    if min(w, h) == 0:
        aspect_ratio = 0
    else:
        aspect_ratio = max(w, h) / min(w, h)

    print("aspect_ratio:", aspect_ratio)
    
    return area, aspect_ratio, box, racio

A função classificacao_peca() usa as informações obtidas de cada contorno para identificar se o objeto é uma peça de Lego, pela sua forma retângular, bem como, classificá-las quanto à sua proporção. 

Os valores foram baseados nos resultados experimentais das imagens fornecidas. Além disso, foi adicionada uma verificação adicional com o objetivo de garantir que o objeto seja grande o suficiente para ser considerado uma peça de lego, evitando a classificação de alguns artefactos.

In [13]:
def classificacao_peca(area, aspect_ratio, racio, min_area):
    
    if area > min_area:

        #area diferenca
        if racio < 89:
            return "indefinido"
    
        #aspect ratio
        if 0.8 < aspect_ratio < 1.4:
            return "2x2"
        elif 1.8 < aspect_ratio < 2.6:
            return"2x4"
        elif 2.8 < aspect_ratio < 3.6:
            return "2x6"
        elif 3.8 < aspect_ratio < 5.1:
            return "2x8"
        else:
            return "indefinido"

# EXECUÇÃO

Para facilitar a execução das funções, seguimos a dica dada pelo professor, que consistia em carregar todas as imagens de teste e armazená-las em um dicionário, evitando assim a necessidade de, sempre que quiséssemos testar o classificador em uma imagem, ter que carregá-las novamente.

temos especificado o caminho até a pasta imagens (pasta_treino), que contém as imagens de treino fornecidas e que, por sua vez, são carregadas pela função carregar_imagens.

Para tornar a execução mais prática, fizemos um ciclo for para percorrer todas as imagens carregadas anteriormente. criamos uma tambem a variável "nome" para obter o nome da imagem correspondente à iteração em que o "for" estiver, permitindo assim acessar o conteúdo dessa imagem e armazená-lo na variável "img".

Apos o armazenamento do conteudo da imagem seguimos para a conversao da mesma, mudando-a de BGR para RGB, essa conversao foi um ponto que o professor acabou por especificar em uma das aulas praticas

depois de termos convertido a imagem, o nosso proximo passo foi a criacao de intervalos inferiores e superiores referentes a cor azul(fundo_inferior e fundo_superior) para podermos posteormente fazer a aplicao da mascara (aplicar_mascara), retirando assim o fundo azul.

Para o conseguirmos dectetar os contornos dos objectos usamos a funcao findContours especificamos os seguintes parametros:
mascara_refinada - mascara melhorada atraves dos operadores morfogicos
cv2.RETR_EXTERNAL - refere a identificacao de contornos externos
cv2.CHAIN_APPROX_SIMPLE - armazena apenas os pontos essenciais

E para desenharmos os contornos usamos a funcao cv2.drawContours() espificiando os seguintes parametros:
imagem_com_contornos - copia da imagem orignal onde os contornos serao desenhados
contornos - É a lista de contornos retornada por cv2.findContours.
-1 - serve para indicar que quer desenhar todos os contornos do array contornos.
(0, 255, 0) - cor dos contornos que serao desenhados
2 - espessura da linha


Tendo completado este passo, avancamos finalmente para a extracao de caracteristicas e classificao.

iniciamos um novo ciclo for para percorrer cada contorno encontrado na mascara.
Dentro deste ciclo, começamos por calcular um retângulo delimitador(bounding box) para cada objecto. (x, y, w, h = cv2.boundingRect(contorno))

onde:
x e y - representam as coordenadas do canto superior esquerdo do objecto.
w e h - correspondem à largura e altura.

Criamos também uma variável "min_area", utilizada para evitar que pequenas imperfeições do fundo sejam classificadas como peças, definimos tambem uma variável margem, usada para evitar que peças encostadas às bordas da imagem sejam consideradas. Essa foi a solução que encontramos para garantir que a peça de lego cinzenta presente na imagem “lego30” fosse ignorada.

A variável "margem" é usada em conjunto com as variáveis "image_h" e "image_w", que armazenam a altura e a largura da imagem. usamos essas variáveis ajudaram-nos a verificar se os lados dos contornos estavam muito próximos das bordas da imagem. Caso não estejam, o objeto segue para a extração das propriedades e  classificação, caso contrário, o objecto é ignorado.

Após a classificação, armazenamos os resultados (tipo, área e aspect_ratio) dentro do array pecas_classificadas, esse array armazena todas as informações das peças apos a classificao. Por fim, escrevemos na imagem os tipos de peças encontrados e gravamos as imagens resultantes na pasta resultados.

In [14]:
pasta_treino = "./imagens"
imagens = carregar_imagens(pasta_treino)
print(len(imagens), "imagens carregadas.")

14 imagens carregadas.


In [15]:
nomes = list(imagens.keys())

for i in range(len(imagens)):
    #selecao da imagem
    nome = nomes[i]
    img = imagens[nome] 

    #conversao para rgb
    rgb_img = converter_para_rgb(img)

    #definicao dos intervalos de cor do fundo
    fundo_inferior = np.array([90, 0, 0])      
    fundo_superior = np.array([255, 255, 255])

    #criacao e aplocacao da mascara
    mascara_fundo = criar_mascara_por_cor(rgb_img, fundo_inferior, fundo_superior)
    mascara_refinada = refinar_mascara(mascara_fundo)
    imagem_isolada = aplicar_mascara(rgb_img, mascara_refinada)


    #desenho dos contornos dos objectos
    contornos, _ = cv2.findContours(mascara_refinada,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    #desenho dos contornos na imagem original
    imagem_com_contornos = img.copy()
    cv2.drawContours(imagem_com_contornos, contornos, -1, (0, 255, 0), 2)
    

    pecas_classificadas = []

    image_h = img.shape[0]
    image_w = img.shape[1]

    min_area = 1500

    for contorno in contornos:
        
        x, y, w, h = cv2.boundingRect(contorno)
        margem = 10
        if x <= margem or y <= margem or x + w >= image_w - margem or y + h >= image_h - margem:
            continue
        else:
            area, aspect_ratio, box, racio = extracao_propriedades(contorno)
            cv2.drawContours(imagem_com_contornos, [box], 0, (255, 0, 0), 2)

            tipo = classificacao_peca(area, aspect_ratio, racio, min_area)

            pecas_classificadas.append((tipo, area, aspect_ratio))

            x, y, w, h = cv2.boundingRect(contorno)
            posicao_texto = (x, y)
            posicao_texto_2 = (x, y - 50)  

            cv2.putText(imagem_com_contornos, tipo, posicao_texto, cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
            #if area > min_area:
                #cv2.putText(imagem_com_contornos, str(aspect_ratio), posicao_texto_2, cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)

    gravar_imagem(cv2.cvtColor(mascara_refinada, cv2.COLOR_GRAY2BGR), f"resultados/mascara_refinada_{i}.jpg")
    gravar_imagem(cv2.cvtColor(imagem_isolada, cv2.COLOR_RGB2BGR), f"resultados/imagem_isolada_{i}.jpg")
    gravar_imagem(imagem_com_contornos, f"resultados/imagem_com_contornos_{i}.jpg")

    print(pecas_classificadas)

aspect_ratio: 1.0185459462757018
aspect_ratio: 1.6981133559283204
aspect_ratio: 2.2287120806750362
aspect_ratio: 1.6901095462486133
aspect_ratio: 2.262927433253323
aspect_ratio: 2.1747574239140945
aspect_ratio: 4.72916654163348
aspect_ratio: 1.0341685015748046
aspect_ratio: 1.0486331140075251
[('2x2', 6258.5, 1.0185459462757018), ('indefinido', 14304.5, 1.6981133559283204), ('2x4', 11897.0, 2.2287120806750362), ('indefinido', 9194.5, 1.6901095462486133), ('2x4', 11580.5, 2.262927433253323), ('2x4', 12463.5, 2.1747574239140945), ('2x8', 23246.0, 4.72916654163348), ('2x2', 6614.0, 1.0341685015748046), ('2x2', 6092.0, 1.0486331140075251)]
aspect_ratio: 2.2030567736474285
aspect_ratio: 1.4093438095871682
aspect_ratio: 2.3090184987144515
aspect_ratio: 4.855026217104629
aspect_ratio: 1.0203001179796636
aspect_ratio: 1.0575540322930164
[('2x4', 11884.5, 2.2030567736474285), ('indefinido', 7019.0, 1.4093438095871682), ('2x4', 12058.5, 2.3090184987144515), ('2x8', 22721.5, 4.855026217104629), (